In [9]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
import numpy as np
!pip install tensorflow
!pip install tflearn
import tflearn



  Running setup.py bdist_wheel for absl-py: started
  Running setup.py bdist_wheel for absl-py: finished with status 'done'
  Stored in directory: C:\Users\gabri\AppData\Local\pip\Cache\wheels\45\07\0e\6880381ca521796cf6cc18ba4ab502c2232e5777099b4df4ae
  Running setup.py bdist_wheel for html5lib: started
  Running setup.py bdist_wheel for html5lib: finished with status 'done'
  Stored in directory: C:\Users\gabri\AppData\Local\pip\Cache\wheels\6f\85\6c\56b8e1292c6214c4eb73b9dda50f53e8e977bf65989373c962
Successfully built absl-py html5lib
  Found existing installation: html5lib 0.999999999
    Uninstalling html5lib-0.999999999:
      Successfully uninstalled html5lib-0.999999999
  Found existing installation: bleach 2.0.0
    Uninstalling bleach-2.0.0:
      Successfully uninstalled bleach-2.0.0
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [10]:
#!pip3 install tensorflow==1.2.0 --ignore-installed
import tensorflow as tf
import random
import json
import string
import unicodedata
import sys


In [13]:

# a table structure to hold the different punctuation used
tbl = dict.fromkeys(i for i in range(sys.maxunicode)
                    if unicodedata.category(chr(i)).startswith('P'))

# method to remove punctuations from sentences.
def remove_punctuation(text):
    return text.translate(tbl)

# initialize the stemmer
stemmer = LancasterStemmer()
# variable to hold the Json data read from the file
data = None


In [14]:

# read the json file and load the training data
with open('data.json') as json_data:
    data = json.load(json_data)
    print(data)



{'time': ['what time is it?', 'how long has it been since we started?', "that's a long time ago", ' I spoke to you last week', ' I saw you yesterday'], 'sorry': ["I'm extremely sorry", 'did he apologize to you?', "I shouldn't have been rude"], 'greeting': ['Hello there!', 'Hey man! How are you?', 'hi'], 'farewell': ['It was a pleasure meeting you', 'Good Bye.', 'see you soon', 'I gotta go now.'], 'age': ["what's your age?", 'How old are you?', "I'm a couple of years older than her", 'You look aged!']}


In [15]:
# get a list of all categories to train for
categories = list(data.keys())
words = []
# a list of tuples with words in the sentence and category name
docs = []

for each_category in data.keys():
    for each_sentence in data[each_category]:
        # remove any punctuation from the sentence
        each_sentence = remove_punctuation(each_sentence)
        print(each_sentence)
        # extract words from each sentence and append to the word list
        w = nltk.word_tokenize(each_sentence)
        print("tokenized words: ", w)
        words.extend(w)
        docs.append((w, each_category))


what time is it
tokenized words:  ['what', 'time', 'is', 'it']
how long has it been since we started
tokenized words:  ['how', 'long', 'has', 'it', 'been', 'since', 'we', 'started']
thats a long time ago
tokenized words:  ['thats', 'a', 'long', 'time', 'ago']
 I spoke to you last week
tokenized words:  ['I', 'spoke', 'to', 'you', 'last', 'week']
 I saw you yesterday
tokenized words:  ['I', 'saw', 'you', 'yesterday']
Im extremely sorry
tokenized words:  ['Im', 'extremely', 'sorry']
did he apologize to you
tokenized words:  ['did', 'he', 'apologize', 'to', 'you']
I shouldnt have been rude
tokenized words:  ['I', 'shouldnt', 'have', 'been', 'rude']
Hello there
tokenized words:  ['Hello', 'there']
Hey man How are you
tokenized words:  ['Hey', 'man', 'How', 'are', 'you']
hi
tokenized words:  ['hi']
It was a pleasure meeting you
tokenized words:  ['It', 'was', 'a', 'pleasure', 'meeting', 'you']
Good Bye
tokenized words:  ['Good', 'Bye']
see you soon
tokenized words:  ['see', 'you', 'soon']
I

In [25]:

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words]
words = sorted(list(set(words)))

#print(words)
print(docs)

[(['what', 'time', 'is', 'it'], 'time'), (['how', 'long', 'has', 'it', 'been', 'since', 'we', 'started'], 'time'), (['thats', 'a', 'long', 'time', 'ago'], 'time'), (['I', 'spoke', 'to', 'you', 'last', 'week'], 'time'), (['I', 'saw', 'you', 'yesterday'], 'time'), (['Im', 'extremely', 'sorry'], 'sorry'), (['did', 'he', 'apologize', 'to', 'you'], 'sorry'), (['I', 'shouldnt', 'have', 'been', 'rude'], 'sorry'), (['Hello', 'there'], 'greeting'), (['Hey', 'man', 'How', 'are', 'you'], 'greeting'), (['hi'], 'greeting'), (['It', 'was', 'a', 'pleasure', 'meeting', 'you'], 'farewell'), (['Good', 'Bye'], 'farewell'), (['see', 'you', 'soon'], 'farewell'), (['I', 'got', 'ta', 'go', 'now'], 'farewell'), (['whats', 'your', 'age'], 'age'), (['How', 'old', 'are', 'you'], 'age'), (['Im', 'a', 'couple', 'of', 'years', 'older', 'than', 'her'], 'age'), (['You', 'look', 'aged'], 'age')]


In [29]:
# create our training data
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(categories)


In [45]:
for doc in docs:
    # initialize our bag of words(bow) for each document in the list
    bow = []
    # list of tokenized words for the pattern
    token_words = doc[0]
    # stem each word
    token_words = [stemmer.stem(word.lower()) for word in token_words]
    # create our bag of words array
    for w in words:
        bow.append(1) if w in token_words else bow.append(0)

    output_row = list(output_empty)
    output_row[categories.index(doc[1])] = 1

    # our training set will contain a the bag of words model and the output row that tells
    # which catefory that bow belongs to.
    training.append([bow, output_row])


In [58]:
# shuffle our features and turn into np.array as tensorflow  takes in numpy array
random.shuffle(training)
training = np.array(training)

# trainX contains the Bag of words and train_y contains the label/ category
train_x = list(training[:, 0])
train_y = list(training[:, 1])


In [73]:

# reset underlying graph data
tf.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=1500, batch_size=8, show_metric=True)
#model.save('model.tflearn')


Training Step: 4499  | total loss: 0.00402 | time: 0.006s
| Adam | epoch: 1500 | loss: 0.00402 - acc: 1.0000 -- iter: 16/19
Training Step: 4500  | total loss: 0.00404 | time: 0.009s
| Adam | epoch: 1500 | loss: 0.00404 - acc: 1.0000 -- iter: 19/19
--


In [82]:
train_y[0]

[0, 0, 0, 1, 0]

In [126]:
def get_tf_record(sentenceL):
    #print(sentenceL)
    global words
    retorno = []
    if type(sentenceL) == list :
        for sentence in sentenceL:
            #print(sentence)
            # tokenize the pattern
            sentence_words = nltk.word_tokenize(sentence)
            # stem each word
            sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
            # bag of words
            bow = [0]*len(words)
            for s in sentence_words:
                for i, w in enumerate(words):
                    if w == s:
                        bow[i] = 1
            retorno.append(bow)

        return(np.array(retorno))
    else:
        sentence = sentenceL
        # tokenize the pattern
        sentence_words = nltk.word_tokenize(sentence)
        # stem each word
        sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
        # bag of words
        bow = [0]*len(words)
        for s in sentence_words:
            for i, w in enumerate(words):
                if w == s:
                    bow[i] = 1
        retorno.append(bow)

        return(np.array(bow))
        

In [100]:
# let's test the mdodel for a few sentences:
# the first two sentences are used for training, and the last two sentences are not present in the training data.
sent_1 = "what time is it?"
sent_2 = "I gotta go now"
sent_3 = "do you know the time now?"
sent_4 = "you must be a couple of years older then her!"

sents = ["what time is it?","I gotta go now","do you know the time now?","you must be a couple of years older then her!"]

In [67]:
print(np.argmax(model.predict([get_tf_record(sent_1)])))
categories

0


['time', 'sorry', 'greeting', 'farewell', 'age']

In [138]:

# we can start to predict the results for each of the 4 sentences
print(categories[np.argmax(model.predict([get_tf_record(sent_1)]))])
print(categories[np.argmax(model.predict([get_tf_record(sent_2)]))])
print(categories[np.argmax(model.predict([get_tf_record(sent_3)]))])
print(categories[np.argmax(model.predict([get_tf_record(sent_4)]))])

time
farewell
farewell
age


In [175]:
x = get_tf_record(sents)
labels = [0,3,3,4]


In [176]:
pred = model.predict(x)

In [180]:
print(pred)
#print(categories[np.argmax(model.predict([get_tf_record(sent_1)]))])
#print(model.predict([get_tf_record(sent_1)]))
#print(np.argmax(get_tf_record([sent_1])))
#print(get_tf_record(sent_1))

[[  9.95620549e-01   2.26564473e-03   9.98753364e-08   2.11372017e-03
    4.43430892e-08]
 [  4.29353713e-05   3.22086047e-09   1.45874766e-03   9.98498321e-01
    8.65566552e-09]
 [  2.90638983e-01   1.47789922e-02   1.36470152e-02   6.79553807e-01
    1.38119771e-03]
 [  9.49604284e-11   1.21153098e-05   9.94933071e-04   8.00174149e-09
    9.98992980e-01]]


In [181]:
pred_y = list(np.argmax(pred,axis=1))
#pred.max(axis=0)
pred_y

[0, 3, 3, 4]

In [185]:
from sklearn.metrics import accuracy_score
accuracy_score(labels,pred_y)
#print(labels, pred_y)

1.0